In [9]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine



In [10]:
# đọc tệp CSV và lưu dữ liệu vào DataFrame
dfOrder = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Order.csv')
dfProduct = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Product.csv')
dfDelivery = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Delivery.csv')
dfDetailOrder = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\DetailOrder.csv',low_memory=Fasle)
dfCustomer = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Customer.csv')
dfCategory = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Category.csv')
dfDepartment = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Department.csv')


C:\Users\June\AppData\Local\Temp\ipykernel_19540\1986651500.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDelivery = pd.read_csv('C:\\Users\\June\\DataWarehouse&BI\\Delivery.csv')


In [45]:
print(dfDelivery.dtypes)
dfDelivery['shipping day'] = dfDelivery['shipping day'].astype(int)
dfDelivery['shipping month'] = dfDelivery['shipping month'].astype(int)
dfDelivery['shipping quarter'] = dfDelivery['shipping quarter'].astype(int)
dfDelivery['shipping year'] = dfDelivery['shipping year'].astype(int)
dfDelivery['Days for shipment (scheduled)'] = dfDelivery['Days for shipment (scheduled)'].astype(int)
dfDelivery['Days for shipping (real)'] = dfDelivery['Days for shipping (real)'].astype(int)
dfDelivery['Late delivery risk'] = dfDelivery['Late delivery risk'].astype(int)
print(dfDelivery.dtypes)

Order Id                           int64
Delivery Status                   object
Shipping Mode                     object
shipping day                     float64
shipping month                   float64
shipping quarter                 float64
shipping year                    float64
Days for shipment (scheduled)    float64
Days for shipping (real)         float64
Late delivery risk               float64
dtype: object


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [11]:
# kết nối đến cơ sở dữ liệu MySQL
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="01062002",
  database="DW"
)

In [4]:
# chèn dữ liệu từ DataFrame vào cơ sở dữ liệu MySQL
try:
    # Create a cursor
    cursor = mydb.cursor()

    # Define the SQL query to insert data into the fact table
    for index, row in dfOrder.iterrows():
    
        sql = "INSERT INTO `Order`(`Type`, `order_status`, `order_id`, `order_day`, `order_month`, `order_quarter`, `order_year`, `order_region`, `order_item_cardprod_id`, `order_state`, `order_country`, `order_city`, `customer_id`) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s)"

        val = (row['Type'], row['Order Status'], row['Order Id'],row['order day'], row['order month'], row['order quarter'],row['order year'], row['Order Region'], row['Order Item Cardprod Id'], row['Order State'],row['Order Country'], row['Order City'], row['Customer Id'])
        cursor.execute(sql, val)
    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")




Data inserted successfully!


In [6]:
try:
    # Create a cursor
    cursor = mydb.cursor()

    for index, row in dfProduct.iterrows():
        sql = "INSERT INTO Product (Product_Name, Product_Price, Product_Card_Id, Category_Id) VALUES (%s, %s, %s, %s)"
        val = (row['Product Name'], row['Product Price'], row['Product Card Id'], row['Category Id'])
        cursor.execute(sql, val)
    

    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")
    


Data inserted successfully!


In [32]:
row_count, column_count = dfDelivery.shape

print(f'Total rows: {row_count}')
print(f'Total columns: {column_count}')

Total rows: 65741
Total columns: 10


In [33]:
# Tìm các hàng có giá trị lỗi
error_rows = dfDelivery[dfDelivery.isna().any(axis=1)]

# In các hàng có giá trị lỗi
print(error_rows)

       Order Id Delivery Status Shipping Mode  shipping day  shipping month  \
15619      2231             NaN           NaN           NaN             NaN   
15620     55165             NaN           NaN           NaN             NaN   
15621      2238             NaN           NaN           NaN             NaN   
15622      4705             NaN           NaN           NaN             NaN   
15623     57211             NaN           NaN           NaN             NaN   
...         ...             ...           ...           ...             ...   
65736     26773             NaN           NaN           NaN             NaN   
65737     26463             NaN           NaN           NaN             NaN   
65738     26383             NaN           NaN           NaN             NaN   
65739     26327             NaN           NaN           NaN             NaN   
65740     26118             NaN           NaN           NaN             NaN   

       shipping quarter  shipping year  Days for sh

In [4]:
try:
    # Create a cursor
    cursor = mydb.cursor()
    dfDelivery.dropna()
    
    cursor.execute("CREATE TABLE delivery (Order_Id INT PRIMARY KEY, Delivery_Status VARCHAR(255), Shipping_Mode VARCHAR(255), Shipping_day INT, Shipping_Month INT, Shipping_Quarter INT, Shipping_Year INT, Days_for_shipment_scheduled INT, Days_for_shipping_real INT, Late_delivery_risk INT)")

    # Define the SQL query to insert data into the fact table
    for index, row in dfDelivery.iterrows():
        sql = "INSERT INTO Delivery (Order_Id, Delivery_Status, Shipping_Mode, Shipping_day, Shipping_Month, Shipping_Quarter, Shipping_Year, Days_for_shipment_scheduled, Days_for_shipping_real, Late_delivery_risk) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val = (row['Order Id'], row['Delivery Status'], row['Shipping Mode'], row['shipping day'], row['shipping month'], row['shipping quarter'], row['shipping year'], row['Days for shipment (scheduled)'], row['Days for shipping (real)'], row['Late delivery risk'])
        cursor.execute(sql, val)
    

    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")
    

Error: 1054 (42S22): Unknown column 'nan' in 'field list'


In [5]:
try:
    # Create a cursor
    cursor = mydb.cursor()
    cursor.execute("CREATE TABLE detailorder (Order_Item_Total FLOAT, Product_Card_Id INT, Sales FLOAT, Order_Item_Id INT, Order_Item_Quantity INT, Order_Id INT , Benefit_per_order FLOAT, Order_Profit_Per_Order FLOAT, Order_Item_Profit_Ratio FLOAT, Order_Item_Product_Price FLOAT)")

    for index, row in dfDetailOrder.iterrows():
        sql = "INSERT INTO DetailOrder (Order_Item_Total, Product_Card_Id, Sales, Order_Item_Id, Order_Item_Quantity, Order_Id, Benefit_per_order, Order_Profit_Per_Order, Order_Item_Profit_Ratio, Order_Item_Product_Price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val = (row['Order Item Total'], row['Product Card Id'], row['Sales'], row['Order Item Id'], row['Order Item Quantity'], row['Order Id'], row['Benefit per order'], row['Order Profit Per Order'], row['Order Item Profit Ratio'], row['Order Item Product Price'])
        cursor.execute(sql, val)
    

    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")
    

Error: 1050 (42S01): Table 'detailorder' already exists


In [27]:
dfCategory.drop_duplicates(keep='first', inplace=True)
dfCustomer.drop_duplicates(keep='first', inplace=True)
dfDepartment.drop_duplicates(keep='first', inplace=True)

In [29]:
try:
    # Create a cursor
    cursor = mydb.cursor()
    cursor.exectue("DROP TABLE customers")
#     cursor.execute("CREATE TABLE customers (customer_id INT AUTO_INCREMENT PRIMARY KEY, customer_state VARCHAR(255), customer_country VARCHAR(255), customer_city VARCHAR(255), customer_street VARCHAR(255), customer_segment VARCHAR(255), customer_lname VARCHAR(255), customer_fname VARCHAR(255))")

    for index, row in dfCustomer.iterrows():
        sql = "INSERT INTO customers (customer_id, customer_state, customer_country, customer_city, customer_street, customer_segment, customer_lname, customer_fname) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        val = (row['Customer Id'], row['Customer State'], row['Customer Country'], row['Customer City'], row['Customer Street'], row['Customer Segment'], row['Customer Lname'], row['Customer Fname'])
        cursor.execute(sql, val)
    

    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")

Error: 1062 (23000): Duplicate entry '20755' for key 'customers.PRIMARY'


In [23]:
try:
    # Create a cursor
    cursor = mydb.cursor()
    cursor.execute("CREATE TABLE category (category_id INT PRIMARY KEY, category_name VARCHAR(255), department_id INT)")
    for index, row in dfCategory.iterrows():
        sql = "INSERT INTO category (category_id, category_name, department_id) VALUES (%s, %s, %s)"
        val = (row["Category Id"], row["Category Name"],row['Department Id'])
        cursor.execute(sql, val)
    

    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")


Data inserted successfully!


In [24]:
try:
    # Create a cursor
    cursor = mydb.cursor()
    cursor.execute("CREATE TABLE department (department_id INT PRIMARY KEY, department_name VARCHAR(255))")

    for index, row in dfDepartment.iterrows():
        sql = "INSERT INTO department (department_id, department_name) VALUES (%s, %s)"
        val = (row['Department Id'], row['Department Name'])
        cursor.execute(sql, val)

    # Commit the changes
    mydb.commit()

    print("Data inserted successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")

Data inserted successfully!


In [ ]:
finally:
    # Close the cursor and connection
    cursor.close()
    mydb.close()